# Agregar conexiones de acceso directo

Originalmente,  las  conexiones  de  acceso  directo  se  propusieron  para  redes  profundas  en  visión  artificial  (específicamente,  en  redes  residuales)  para  mitigar  el  desafío  de  los  gradientes  evanescentes.
El  problema  del  gradiente  que  desaparece  se  refiere  al  problema  en  el  cual  los  gradientes  (que  guían  las  actualizaciones  de  peso  durante  el  entrenamiento)  se  vuelven  progresivamente  más  pequeños  a  medida  que  se  propagan  hacia  atrás  a  través  de  las  capas,  lo  que  dificulta  el  entrenamiento  efectivo  de  las  capas  anteriores.

![Texto alternativo](./imgs/4.12.png)

Comparación  entre  una  red  neuronal  profunda  de  5  capas  sin  conexiones  de  acceso  directo  (izquierda)  y  con  conexiones  de  acceso  directo  (derecha).  Las  conexiones  de  acceso  directo  implican  sumar  las  entradas  de  una  capa  a  sus  salidas,  creando  así  una  ruta  alternativa  que  omite  ciertas  capas.

Una  conexión  de  acceso  directo  crea  una  ruta  alternativa  más  corta  para  que  el  gradiente  fluya  a  través  de  la  red  saltando  una  o  más  capas.  Esto  se  logra  sumando  la  salida  de  una  capa  a  la  de  una  capa  posterior.  
Por  eso,  estas  conexiones  también  se  conocen  como  conexiones  de  salto.  Desempeñan  un  papel  crucial  en  la  preservación  del  flujo  de  gradientes  durante  el  paso  hacia  atrás  en  el  entrenamiento.
````
x ───────────────┐
                 ▼
              [Bloque]
                 ▼
             salida = Bloque(x) + x


In [1]:
import torch
import torch.nn as nn
class GELU(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, x):
        return 0.5 * x * (
            1 + torch.tanh(torch.sqrt(torch.tensor(2.0 / torch.pi)) * (x + 0.044715 * torch.pow(x, 3))
        ))
class ExampleDeepNeuralNetwork(nn.Module):
    def __init__(self, layer_sizes, use_shortcut):
        super().__init__()
        self.use_shortcut = use_shortcut
        self.layers = nn.ModuleList([
            #Implementación de 5 capas
            nn.Sequential(nn.Linear(layer_sizes[0], layer_sizes[1]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[1], layer_sizes[2]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[2], layer_sizes[3]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[3], layer_sizes[4]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[4], layer_sizes[5]), GELU())
        ])
    def forward(self, x):
        for layer in self.layers:
            #salida de la capa actual
            layer_output = layer(x)
            #mirar si se puede ahcer shortcut
            if self.use_shortcut and x.shape == layer_output.shape:
                x = x + layer_output
            else:
                x = layer_output
        return x

El  código  implementa  una  red  neuronal  profunda  con  cinco  capas,  cada  una  compuesta  por  una  capa  lineal  y  una  función  de  activación  GELU .  En  el  paso  directo,  pasamos  iterativamente  la  entrada  a  través  de  las  capas  y,  opcionalmente,  añadimos  las  conexiones  de  acceso  directo

In [5]:
#Red neuronal sin conexiones de accesos directo

layer_sizes = [3, 3, 3, 3, 3, 1]
sample_input = torch.tensor([[1., 0., -1.]])
torch.manual_seed(123)
model_without_shortcup = ExampleDeepNeuralNetwork(layer_sizes=layer_sizes, use_shortcut=False)

#Función que calcula los gradientes en la regresión del modelo
def print_gradients(model, x):
    #Forward pass
    output = model(x)
    target = torch.tensor([[0.]])

    #Calcular la perdida de como de cerca del target la salida está
    loss = nn.MSELoss()
    loss = loss(output, target)

    #Backward pass para calcular gradientes
    loss.backward()

    for name, param in model.named_parameters():
        if 'weight' in name:
            print(f"{name} has gradient mean of {param.grad.abs().mean().item()}")
print_gradients(model_without_shortcup, sample_input)

layers.0.0.weight has gradient mean of 0.00020173587836325169
layers.1.0.weight has gradient mean of 0.0001201116101583466
layers.2.0.weight has gradient mean of 0.0007152041653171182
layers.3.0.weight has gradient mean of 0.001398873864673078
layers.4.0.weight has gradient mean of 0.005049646366387606


En  el  código  anterior,  se especifica  una  función  de  pérdida  que  calcula  la  proximidad  entre  la  salida  del  modelo  y  un  objetivo  especificado  por  el  usuario  (aquí,  para  simplificar,  el  valor  0).  Luego,  al  llamar  a  loss.backward(),  PyTorch  calcula  el  gradiente  de  pérdida  para  cada  capa  del  modelo.  Podemos  iterar  
sobre  los  parámetros  de  peso  mediante  model.named_parameters().  Supongamos  que  tenemos  una  matriz  de  parámetros  de  peso  de  3×3  para  una  capa  dada.  En  ese  caso,  esta  capa  tendrá  valores  de  gradiente  de  3×3,  e  imprimimos  el  gradiente  absoluto  medio  de  estos  valores  de  gradiente  de  3×3  para  
obtener  un  único  valor  de  gradiente  por  capa  y  comparar  los  gradientes  entre  capas  con  mayor  facilidad.
En  resumen,  el  método .backward()  es  un  método  práctico  en  PyTorch  que  calcula  los  gradientes  de  pérdida,  necesarios  durante  el  entrenamiento  del  modelo,  sin  tener  que  implementar  los  cálculos  matemáticos  para  el  gradiente,  lo  que  facilita  enormemente  el  trabajo  con  redes  neuronales  profundas. 

Los  gradientes  se  vuelven  más  pequeños  a  medida  que  avanzamos  desde  la  última  capa  (capas.4)  a  la  primera  capa  (capas.0),  lo  que  es  un  fenómeno  llamado  el  problema  del  gradiente  que  desaparece.

In [6]:
#Red neuronal con conexiones directas
torch.manual_seed(123)
model_with_shortcut = ExampleDeepNeuralNetwork(
                        layer_sizes, use_shortcut=True
                        )
print_gradients(model_with_shortcut, sample_input)

layers.0.0.weight has gradient mean of 0.22169792652130127
layers.1.0.weight has gradient mean of 0.20694106817245483
layers.2.0.weight has gradient mean of 0.32896995544433594
layers.3.0.weight has gradient mean of 0.2665732502937317
layers.4.0.weight has gradient mean of 1.3258541822433472


Como se puede ver,  según  el  resultado,  la  última  capa  (capas.4)  aún  tiene  un  gradiente  mayor  que  las  demás.  Sin  embargo,  el  valor  del  gradiente  se  estabiliza  a  medida  que  avanzamos  hacia  la  primera  capa  (capas.0)  y  no  se  reduce  a  un  valor  extremadamente  pequeño.

En  conclusión,  las  conexiones  de  acceso  directo  son  importantes  para  superar  las  limitaciones  que  plantea  el  problema  del  gradiente  evanescente  en  redes  neuronales  profundas.  Las  conexiones  de  acceso  directo  son  un  componente  fundamental  de  modelos  muy  grandes,  como  los  LLM,  y  facilitarán  un  entrenamiento  más  eficaz  al  garantizar  un  flujo  de  gradiente  consistente  entre  capas  cuando  entrenemos  el  modelo  GPT  en  el  siguiente  capítulo.

En la siguiente sección se conectarán todos los pasos hasta ahora descritos (normalización, GELU, modulo de avance y conexiones de acceso directo) en un bloque de transformador.

[Conexión de capas de atención y capas lineales en un bloque transformador](./5_capas_atencion_lineales_transformador.ipynb)